This program uses semaphores and threads to calculate the sum up to a passed value using a single producer and consumer in C

In [1]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    printf("Producer created\n");
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    printf("Consumer created\n");
    int sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum is %d\n", numIters, sum);
}

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    pthread_t pid, cid;           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);
    pthread_create(&pid, &attr, Producer, NULL);
    pthread_create(&cid, &attr, Consumer, NULL);
    pthread_join(pid, NULL);
    pthread_join(cid, NULL);
}

Overwriting pc.c


In [2]:
!gcc pc.c -lpthread -o pc

In [3]:
!./pc 5

Consumer created
Producer created
For 5 iterations, the sum is 10


This program uses semaphores and threads to calculate the sum up to a passed value and the individual sum of each consumer using an arbitrary number of producers and consumers in C

In [9]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    printf("Producer %d created\n", *(int*)arg);
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    printf("Consumer %d created\n", *(int*)arg);
    int* sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum of consumer %d is %d\n", numIters, *(int*)arg, sum);
    return (void *) sum;
}

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    int totalSum = 0;
    int numProdCon = atoi(argv[2]);
    
    pthread_t producers[numProdCon];
    pthread_t consumers[numProdCon];           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);
    
    int ids[numProdCon];
    for (int i = 0; i < numProdCon; i++) ids[i] = i;
    
    for (int i = 0; i < numProdCon; i++) pthread_create(&producers[i], &attr, Producer, &ids[i]);
    for (int i = 0; i < numProdCon; i++) pthread_create(&consumers[i], &attr, Consumer, &ids[i]);
    
    for (int i = 0; i < numProdCon; i++) pthread_join(producers[i], NULL);
    for (int i = 0; i < numProdCon; i++) {
        int res; 
        pthread_join(consumers[i], (void **) &res);
        totalSum += res;
    }
    
    printf("The expected total sum is %d\n", numIters * (numIters - 1) / 2 * numProdCon);
    printf("The computed total sum is %d", totalSum);
}

Overwriting pc.c


In [10]:
!gcc pc.c -lpthread -o pc

In [11]:
!./pc 10 5

Producer 1 created
Consumer 0 created
Producer 3 created
Consumer 1 created
Consumer 2 created
Producer 0 created
Producer 4 created
Consumer 4 created
Consumer 3 created
Producer 2 created
For 10 iterations, the sum of consumer 4 is 144
For 10 iterations, the sum of consumer 1 is 168
For 10 iterations, the sum of consumer 0 is 192
For 10 iterations, the sum of consumer 2 is 128
For 10 iterations, the sum of consumer 3 is 268
The expected total sum is 225
The computed total sum is 900